In [12]:
# Basic import necessary for configuration.
import os
import warnings
import requests

warnings.simplefilter("ignore", RuntimeWarning)

maindir = os.path.join(os.getcwd(), '../Data/')

# Set CRDS cache directory to user home if not already set.
if os.getenv('CRDS_PATH') is None:
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds_cache')

# Check whether the CRDS server URL has been set. If not, set it.
if os.getenv('CRDS_SERVER_URL') is None:
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Output the current CRDS path and server URL in use.
print('CRDS local filepath:', os.environ['CRDS_PATH'])
print('CRDS file server:', os.environ['CRDS_SERVER_URL'])

CRDS local filepath: /home/fran/crds_cache
CRDS file server: https://jwst-crds.stsci.edu


In [13]:
# ----------------------General Imports----------------------
import time
import glob
import json
import itertools
import numpy as np
import pandas as pd

# --------------------Astroquery Imports---------------------
from astroquery.mast import Observations

# ----------------------Astropy Imports----------------------
# Astropy utilities for opening FITS files, downloading demo files, etc.
from astropy.table import Table
from astropy.stats import sigma_clip
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch
from astropy.visualization import LinearStretch, AsinhStretch, simple_norm

# ----------------------Plotting Imports---------------------
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

# -------------------File Download Imports-------------------
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial

In [14]:
# ----------------------JWST Calibration Pipeline Imports----------------------
import jwst  # Import the base JWST and CRDS packages.
import crds
from crds.client import api
from stpipe import crds_client

# JWST pipelines (each encompassing many steps).
from jwst.pipeline import Detector1Pipeline  # calwebb_detector1
from jwst.pipeline import Spec2Pipeline  # calwebb_spec2
from jwst.pipeline import Tso3Pipeline  # calwebb_tso3
from jwst.extract_1d import Extract1dStep  # Extract1D Step

# JWST pipeline utilities
from jwst import datamodels  # JWST pipeline utilities: datamodels.
from jwst.associations import asn_from_list  # Tools for creating association files.
from jwst.associations.lib.rules_level2b import Asn_Lv2SpecTSO
from jwst.associations.lib.rules_level3 import DMS_Level3_Base

# Check the default context for the Pipeline version
default_context = crds.get_default_context('jwst', state='build')
print("JWST Calibration Pipeline Version = {}".format(jwst.__version__))
print(f"Default CRDS Context for JWST Version {jwst.__version__}: {default_context}")
print(f"Using CRDS Context: {os.environ.get('CRDS_CONTEXT', default_context)}")

JWST Calibration Pipeline Version = 1.18.0
Default CRDS Context for JWST Version 1.18.0: jwst_1364.pmap
Using CRDS Context: jwst_1364.pmap


In [15]:
sci_dir = os.path.join(maindir, f'data_01366/Obs003/')
uncal_dir = os.path.join(sci_dir, f'uncal/')
det1_dir = os.path.join(sci_dir, 'stage1/') 

os.makedirs(det1_dir, exist_ok=True)

In [16]:
# -------------------------Boilerplate dictionary setup-------------------------
det1dict = {}
det1dict['group_scale'], det1dict['dq_init'], det1dict['saturation'] = {}, {}, {}
det1dict['superbias'], det1dict['refpix'] = {}, {}
det1dict['linearity'], det1dict['dark_current'], det1dict['jump'] = {}, {}, {}
det1dict['clean_flicker_noise'], det1dict['ramp_fit'] = {}, {}
det1dict['gain_scale'] = {}

# -----------------------------Set step parameters------------------------------

# Overrides for whether or not certain steps should be skipped (example).
det1dict['linearity']['skip'] = False  # This is the default.

# Turn on multi-core processing for jump step (off by default).
# Choose what fraction of cores to use (quarter, half, or all).
det1dict['jump']['maximum_cores'] = 'half'

# Turn on/off detection of cosmic ray snowballs (on by default).
det1dict['jump']['expand_large_events'] = False

In [17]:
# list files in uncal directory
uncal_files = glob.glob(uncal_dir + '*nrs1_uncal.fits')
print(f"Uncalibrated files found: {len(uncal_files)}")
print(f"Science UNCAL Files:\n{'-'*20}\n" + "\n".join(uncal_files))

Uncalibrated files found: 4
Science UNCAL Files:
--------------------
/home/fran/OneDrive/BSc Project/Notebooks/../Data/data_01366/Obs003/uncal/jw01366003001_04101_00001-seg003_nrs1_uncal.fits
/home/fran/OneDrive/BSc Project/Notebooks/../Data/data_01366/Obs003/uncal/jw01366003001_04101_00001-seg002_nrs1_uncal.fits
/home/fran/OneDrive/BSc Project/Notebooks/../Data/data_01366/Obs003/uncal/jw01366003001_04101_00001-seg001_nrs1_uncal.fits
/home/fran/OneDrive/BSc Project/Notebooks/../Data/data_01366/Obs003/uncal/jw06491001001_04101_00001-seg001_nrs1_uncal.fits


In [18]:
# --------------------------Science UNCAL files--------------------------
for uncal_file in sorted(glob.glob(uncal_dir + '*uncal.fits')):

    print(f"Applying Stage 1 Corrections & Calibrations to: "
            f"{os.path.basename(uncal_file)}")

    det1_result = Detector1Pipeline.call(uncal_file,
                                            save_results=True,
                                            steps=det1dict,
                                            output_dir=det1_dir)
print("Stage 1 has been completed for SCI data! \n")

Applying Stage 1 Corrections & Calibrations to: jw01366003001_04101_00001-seg001_nrs1_uncal.fits


2025-05-17 17:11:33,171 - CRDS - INFO -  Calibration SW Found: jwst 1.18.0 (/home/fran/.local/lib/python3.13/site-packages/jwst-1.18.0.dist-info)
2025-05-17 17:11:33,681 - stpipe - INFO - PARS-JUMPSTEP parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0003.asdf
2025-05-17 17:11:33,692 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0003.asdf
2025-05-17 17:11:33,713 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-05-17 17:11:33,713 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-05-17 17:11:33,714 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-05-17 17:11:33,715 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-05-17 17:11:33,716 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-05-17 17:11:33,71

Applying Stage 1 Corrections & Calibrations to: jw01366003001_04101_00001-seg001_nrs2_uncal.fits


2025-05-17 17:14:14,574 - stpipe - INFO - PARS-JUMPSTEP parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0003.asdf
2025-05-17 17:14:14,596 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0003.asdf
2025-05-17 17:14:14,617 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-05-17 17:14:14,618 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-05-17 17:14:14,619 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-05-17 17:14:14,620 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-05-17 17:14:14,621 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-05-17 17:14:14,622 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2025-05-17 17:14:14,622 - stpipe.Detector1Pipeline.superbias - INFO - SuperBia

Applying Stage 1 Corrections & Calibrations to: jw01366003001_04101_00001-seg002_nrs1_uncal.fits


...

Applying Stage 1 Corrections & Calibrations to: jw01366003001_04101_00001-seg003_nrs2_uncal.fits


2025-05-17 17:24:52,717 - stpipe - INFO - PARS-JUMPSTEP parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0003.asdf
2025-05-17 17:24:52,727 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0003.asdf
2025-05-17 17:24:52,747 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-05-17 17:24:52,748 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-05-17 17:24:52,749 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-05-17 17:24:52,750 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-05-17 17:24:52,751 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-05-17 17:24:52,752 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2025-05-17 17:24:52,753 - stpipe.Detector1Pipeline.superbias - INFO - SuperBia

Applying Stage 1 Corrections & Calibrations to: jw06491001001_04101_00001-seg001_nrs1_uncal.fits


2025-05-17 17:27:32,342 - stpipe - INFO - PARS-JUMPSTEP parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0003.asdf
2025-05-17 17:27:32,353 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /home/fran/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0003.asdf
2025-05-17 17:27:32,382 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-05-17 17:27:32,383 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-05-17 17:27:32,384 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-05-17 17:27:32,385 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-05-17 17:27:32,386 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-05-17 17:27:32,387 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2025-05-17 17:27:32,388 - stpipe.Detector1Pipeline.superbias - INFO - SuperBia

Stage 1 has been completed for SCI data! 

